# Import libraries

In [1]:
import warnings
warnings.filterwarnings( 'ignore' )

# Basic libraries
import os
import pickle
import numpy    as np
import xgboost

# Data

In [2]:
# Load data
X = np.load('Data/data.npz', allow_pickle=True)['X']
y = np.load('Data/data.npz', allow_pickle=True)['y']

# Label encoding for targe class
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
pickle.dump(le, open('Model/Label_encoder.pkl', 'wb'))

## Prediction model

In [3]:
# Parameters
params = {
    'n_estimators'      : 800,
    'learning_rate'     : 1e-1,
    'max_depth'         : 10,
    'reg_alpha'         : 20, 
    'reg_lambda'        : 1.0, 
    'gamma'             : 1, 
    'min_child_weight'  : 2, 
    'max_leaves'        : 2, 
    'eval_metric'       : ['auc', 'aucpr', 'mlogloss']
}


# Setup model
model = xgboost.XGBClassifier(objective           = 'multi:softmax',  
                              n_jobs              = -1,                                   
                              validate_parameters = True, 
                              verbosity           = 1,
                              tree_method         = 'hist',
                              **params)

# Train model
weights = [y.shape[0] / np.where(y == i)[0].shape[0] for i in np.unique(y)]

model.fit(X, y,
          eval_set = [ (X, y) ],
          sample_weight = [weights[int(x)] for x in y],
          verbose = 50);

[0]	validation_0-auc:0.58062	validation_0-aucpr:0.16563	validation_0-mlogloss:2.00307
[50]	validation_0-auc:0.72916	validation_0-aucpr:0.61725	validation_0-mlogloss:1.00277
[100]	validation_0-auc:0.76837	validation_0-aucpr:0.67483	validation_0-mlogloss:0.82584
[150]	validation_0-auc:0.78600	validation_0-aucpr:0.70294	validation_0-mlogloss:0.74900
[200]	validation_0-auc:0.79545	validation_0-aucpr:0.71742	validation_0-mlogloss:0.70725
[250]	validation_0-auc:0.80221	validation_0-aucpr:0.72691	validation_0-mlogloss:0.67992
[300]	validation_0-auc:0.80782	validation_0-aucpr:0.73465	validation_0-mlogloss:0.65898
[350]	validation_0-auc:0.81204	validation_0-aucpr:0.74021	validation_0-mlogloss:0.64326
[400]	validation_0-auc:0.81569	validation_0-aucpr:0.74518	validation_0-mlogloss:0.63078
[450]	validation_0-auc:0.81880	validation_0-aucpr:0.75021	validation_0-mlogloss:0.62034
[500]	validation_0-auc:0.82147	validation_0-aucpr:0.75353	validation_0-mlogloss:0.61183
[550]	validation_0-auc:0.82398	vali

In [4]:
pickle.dump(model, open('Model/model.pkl', 'wb'))